In [1]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cufft-cu12 11.3.3.83 which is incompatible.
torch 2.5.1+cu124 requires nvidia-curand-cu12==10.3.5.147; platform_system == "Linux" and platform_machin

In [2]:
#Not used
from datasets import load_dataset

#Imported directly from kaggle
import pandas as pd

df = pd.read_csv("/kaggle/input/recipes/dataset/full_dataset.csv")

In [3]:
df.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [4]:
dim_dataset = len(df)

print(f"In the dataset there are {dim_dataset} recipe. \n")

In the dataset there are 2231142 recipe. 



**Analysis first column: Unnamed: 0**

In [5]:
fields = list(df.columns)
    
first_cols = list(df.columns[:1])  
duplicates = df[df.duplicated(subset=first_cols)]
  
print(f"Duplicates index: {duplicates}")

Duplicates index: Empty DataFrame
Columns: [Unnamed: 0, title, ingredients, directions, link, source, NER]
Index: []


“Unnamed: 0” corresponds to the ‘id’ field, which uniquely identifies a prescription in the database

In [6]:
import ast 


for field in fields:
    sample_value = df[field].dropna().iloc[0]  
    
    if isinstance(sample_value, str):
        try:
            evaluated = ast.literal_eval(sample_value)
            sample_value = evaluated
        except (ValueError, SyntaxError):
            pass 

    print(f"\nField: {field}")
    print(f"Type: {type(sample_value).__name__}")
    
    # If it's a list or dict, show inner types
    if isinstance(sample_value, list):
        inner_types = set(type(x).__name__ for x in sample_value)
        print(f"Inner types in list: {inner_types}")
    elif isinstance(sample_value, dict):
        key_types = set(type(k).__name__ for k in sample_value.keys())
        value_types = set(type(v).__name__ for v in sample_value.values())
        print(f"Inner types in dict - keys: {key_types}, values: {value_types}")


Field: Unnamed: 0
Type: int64

Field: title
Type: str

Field: ingredients
Type: list
Inner types in list: {'str'}

Field: directions
Type: list
Inner types in list: {'str'}

Field: link
Type: str

Field: source
Type: str

Field: NER
Type: list
Inner types in list: {'str'}


**Analysis second column: title**

In [7]:
from collections import defaultdict

second_col = df.columns[1]  
upper_title = df[second_col].str.upper()

title_indices = defaultdict(list)

for i, title in enumerate(upper_title):
    title_indices[title].append(i)

duplicates_dict = {title: idxs for title, idxs in title_indices.items() if len(idxs) > 1}

l = []
for i, (title, idxs) in enumerate(duplicates_dict.items()):
    l.append(title)
    if i >= 4:  # Stop after 5 examples
        break

print(f"Example of duplicates: {l} \n")

single_recipe = {title: idxs for title, idxs in title_indices.items() if len(idxs) == 1}

count = 0
maxR = 1
maxItem = ""
for item in duplicates_dict:
    #print(f"Item: {item}, recipes: {len(duplicates_dict[item])}")
    count += len(duplicates_dict[item])
    if len(duplicates_dict[item]) > maxR:
        maxR = len(duplicates_dict[item])
        maxItem = item

print(f"Number of title: {len(title_indices)}. \n")

print(f"Number of title with one recipe: {len(single_recipe)}, with more than one: {len(title_indices) - len(single_recipe)}. \n")

print(f"MAX number for recipe: {maxR}, {maxItem}. \n")

print(f"AVG recipe for title: {count/len(duplicates_dict)} (counting only duplicates). \n")

print(f"AVG recipe for title: {count/len(title_indices)} . \n")


Example of duplicates: ['CREAMY CORN', 'CHEESEBURGER POTATO SOUP', 'RHUBARB COFFEE CAKE', 'SCALLOPED CORN', 'MILLIONAIRE PIE'] 

Number of title: 1297133. 

Number of title with one recipe: 1134371, with more than one: 162762. 

MAX number for recipe: 4106, CHICKEN CASSEROLE. 

AVG recipe for title: 6.738495471916049 (counting only duplicates). 

AVG recipe for title: 0.8455347292837357 . 



Now let's take some time to understand how recipes with the same name differ, focusing only on the ingredients 

In [8]:

ingredients = df.columns[6]

#Insert a duplicates recipe from the dataset
recipe = "CHICKEN CASSEROLE"

setIngr = set()
indexRecipe = duplicates_dict[recipe]

print(f"Example of index: {indexRecipe[:10]} \n")
print(f"Recipe ing: {df.iloc[63][ingredients]} \n")

ingredientsCounter = defaultdict(int) 

#Some statistics 
counter = 0
for idx in indexRecipe:
    counter += 1
    ingredientList_str = df.iloc[idx][ingredients].upper()
    ingredientList = ast.literal_eval(ingredientList_str)
    for ingredient in ingredientList:
        if ingredient in ingredientsCounter:
            ingredientsCounter[ingredient] += 1
        else:
            ingredientsCounter[ingredient] = 1

nPrint = 10
numRecipe = len(indexRecipe)
print(f"Percentage of ingredients in {recipe}: \n")
for ingr, count in ingredientsCounter.items():
    print(f"{ingr}:  {count/numRecipe:.2%}")
    nPrint -= 1
    if nPrint == 0:
        break



Example of index: [63, 131, 356, 428, 539, 653, 990, 1009, 1138, 1381] 

Recipe ing: ["cream of mushroom soup", "cream of chicken soup", "cream of celery soup", "chicken", "butter", "rice"] 

Percentage of ingredients in CHICKEN CASSEROLE: 

CREAM OF MUSHROOM SOUP:  30.08%
CREAM OF CHICKEN SOUP:  60.01%
CREAM OF CELERY SOUP:  16.76%
CHICKEN:  57.01%
BUTTER:  20.29%
RICE:  13.96%
ONION SOUP:  2.78%
CREAM OF CHICKEN:  1.88%
MUSHROOMS:  7.77%
BITE:  0.05%


In [9]:
# Sort ingredientsCounter by count in descending order
sorted_ingredients = sorted(ingredientsCounter.items(), key=lambda x: x[1], reverse=True)

nPrint = 10
numRecipe = len(indexRecipe)
print(f"Percentage of ingredients in {recipe}: \n")
for ingr, count in sorted_ingredients:
    print(f"{ingr}: {count/numRecipe:.2%}")  
    nPrint -= 1
    if nPrint == 0:
        break

Percentage of ingredients in CHICKEN CASSEROLE: 

CREAM OF CHICKEN SOUP: 60.01%
CHICKEN: 57.01%
CHICKEN BREASTS: 35.39%
CREAM OF MUSHROOM SOUP: 30.08%
ONION: 29.83%
SOUR CREAM: 23.16%
MARGARINE: 22.50%
CHICKEN BROTH: 22.36%
MILK: 21.36%
BUTTER: 20.29%


The recipe duplicated several times has only 60% more ingredient present. The main problem is also in the names, maybe the same ingredient with different names

In [10]:
del sorted_ingredients, ingredientsCounter, duplicates_dict, upper_title, first_cols

**Analysis second column: Directions**

In [11]:
directions_col = df['directions'].dropna().apply(ast.literal_eval)

In [12]:
step_counts = directions_col.apply(lambda x: len(x) if isinstance(x, list) else 0)

print("Analysis of 'directions' field:\n")
print(f"Total recipes with directions: {len(step_counts)}")
print(f"Minimum steps in a recipe: {step_counts.min()}")
print(f"Maximum steps in a recipe: {step_counts.max()}")
print(f"Average number of steps: {step_counts.mean():.2f}")
print(f"Median number of steps: {step_counts.median()}")

Analysis of 'directions' field:

Total recipes with directions: 2231142
Minimum steps in a recipe: 1
Maximum steps in a recipe: 164
Average number of steps: 6.61
Median number of steps: 5.0


In [13]:
import numpy as np

all_instructions = [instruction for instructions in directions_col for instruction in instructions]

instruction_lengths = [len(instr) for instr in all_instructions]

print("\nAnalysis of single instructions:\n")
print(f"Total instructions: {len(instruction_lengths)}")
print(f"Shortest instruction length: {np.min(instruction_lengths)} characters")
print(f"Longest instruction length: {np.max(instruction_lengths)} characters")
print(f"Average instruction length: {np.mean(instruction_lengths):.2f} characters")
print(f"Median instruction length: {np.median(instruction_lengths):.2f} characters")


Analysis of single instructions:

Total instructions: 14755943
Shortest instruction length: 0 characters
Longest instruction length: 5406 characters
Average instruction length: 72.14 characters
Median instruction length: 52.00 characters


In [14]:
del directions_col, all_instructions, instruction_lengths

**Analysis second column: Link & source**

I leave out the recipes that come from the old dataset and dwell on those with a link

In [15]:
# Select columns
col4 = df.iloc[:, 4]  
col5 = df.iloc[:, 5]  

mask = col5 != "Gathered"
filtered_col5 = col5[mask]
print(filtered_col5)

1643098    Recipes1M
1643099    Recipes1M
1643100    Recipes1M
1643101    Recipes1M
1643102    Recipes1M
             ...    
2231137    Recipes1M
2231138    Recipes1M
2231139    Recipes1M
2231140    Recipes1M
2231141    Recipes1M
Name: source, Length: 588044, dtype: object


In [16]:
mask = ~col5.str.contains("Recipes1M", na=False)  # "~" means NOT, na=False to avoid NaN problems
filtered_col4 = col4[mask]
filtered_col4_inv = col4[~mask]

print(f"Total entries in filtered column link: {len(filtered_col4)}\n")

def extract_base_link(link):
    if pd.isna(link):
        return None
    return link.split('/')[0]

base_links = filtered_col4.apply(extract_base_link)

base_link_counts = base_links.value_counts()

print("Top base links:\n")
print(base_link_counts.head(10))

print(f"\n\n Old dataset: {len(filtered_col4_inv)}")

Total entries in filtered column link: 1643098

Top base links:

link
www.cookbooks.com      896341
www.food.com           290565
www.epicurious.com      94398
www.myrecipes.com       64862
www.allrecipes.com      61398
www.yummly.com          51963
www.tasteofhome.com     51594
tastykitchen.com        50320
food52.com              48501
recipes-plus.com        20524
Name: count, dtype: int64


 Old dataset: 588044


**Analysis second column: NER**

In [17]:
from collections import Counter

ingredienti_unici = set()
ingredient_counter = Counter()

for ingr_str in df['NER']:
    try:
        lista_ingredienti = ast.literal_eval(ingr_str)
        for ingrediente in lista_ingredienti:
            ingrediente_pulito = ingrediente.lower().strip()
            ingredienti_unici.add(ingrediente_pulito)
            ingredient_counter[ingrediente_pulito] += 1
    except:
        continue

print(f"Total number of unique ingredients: {len(ingredienti_unici)}\n")

print("Top 10 ingredients by frequency:")
for ingrediente, count in ingredient_counter.most_common(10):
    print(f"{ingrediente}: {count} times")

Total number of unique ingredients: 198899

Top 10 ingredients by frequency:
salt: 1013713 times
sugar: 662836 times
butter: 539979 times
flour: 488089 times
eggs: 422213 times
onion: 391271 times
garlic: 388187 times
milk: 370287 times
water: 351248 times
vanilla: 293882 times


**vocabulary analysis**

How big is the vocabulary of the collection? How big is the vocabulary of a document on average? 

While the analysis on ingredients has already been done, the vocabulary for recipe instructions has not. Let us therefore try to extract some useful information.

In [18]:
import nltk
import pandas as pd
import ast

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
from nltk.corpus import stopwords
print('English stopwords:')
print(stopwords.words('english'))

English stopwords:
['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'sh

In [20]:
#Flatten the list of instraction
directions_col = df['directions'].dropna().apply(ast.literal_eval)
all_instructions_list = [instruction for instructions in directions_col for instruction in instructions]

From here on I work on a subset because the full dataset is too heavy, subset extracted completely randomly

In [21]:
import re
import random
from nltk.corpus import stopwords
from nltk.probability import FreqDist

division = 1000

subset_size = len(all_instructions_list) // division  
subset_instructions = random.sample(all_instructions_list, subset_size)

full_text = ' '.join(subset_instructions)

full_text = full_text.lower()
full_text = re.sub(r'[^a-zA-Z0-9\s]', '', full_text)  # keep only letters, numbers, spaces

words = full_text.split()

words_nostopwords = [w for w in words if w not in stopwords.words('english')]

fdist = FreqDist(words_nostopwords)

print("Top 10 most common words:")
for word, freq in fdist.most_common(10):
    print(f"{word}: {freq}")


Top 10 most common words:
add: 2708
minutes: 2474
heat: 1591
mix: 1153
mixture: 1134
pan: 1128
stir: 1119
cook: 1090
bowl: 1080
1: 1064


In [22]:
#Find emoticons, if any

emoticon_regex = '(\:\w+\:|\<[\/\\]?3|[\(\)\\\D|\*\$][\-\^]?[\:\;\=]|[\:\;\=B8][\-\^]?[3DOPp\@\$\*\\\)\(\/\|])(?=\s|[\!\.\?]|$)'
emoticons_in_pos = re.findall(emoticon_regex,full_text)
counts = nltk.FreqDist(emoticons_in_pos)
print(f"Number of emoticons: {counts}")

Number of emoticons: <FreqDist with 1 samples and 1 outcomes>


In [23]:
#